In [1]:
import numpy as np
import pretty_midi
import music21 as m21
import os
import copy
import Levenshtein as lev
from io import StringIO
import sys
import pickle
import random

In [2]:
class Measure:
    def __init__(self, measure, instrument):
        self.measure = measure
        self.instrument = instrument
    
    def __repr__(self):
        return f'Measure of {self.instrument}'

In [3]:
path ='../game/data/base'

In [4]:
# read a midi file and load it into a music21 object
def load(file_path):
    return m21.converter.parse(os.path.join(path, file_path))

In [5]:
songs = []
for file in os.listdir(path):
    print(file)
    songs.append(load(file))
    break

4-17 - Summoned Beast Battle (extended).mid


In [6]:
orig_song = songs[0]

In [7]:
midi_tracks = m21.midi.translate.streamHierarchyToMidiTracks(orig_song)
midi_tracks

[<music21.midi.MidiTrack 0 -- 6 events>,
 <music21.midi.MidiTrack 1 -- 14466 events>,
 <music21.midi.MidiTrack 2 -- 15362 events>,
 <music21.midi.MidiTrack 3 -- 3410 events>,
 <music21.midi.MidiTrack 4 -- 5858 events>,
 <music21.midi.MidiTrack 5 -- 154 events>,
 <music21.midi.MidiTrack 6 -- 2986 events>,
 <music21.midi.MidiTrack 7 -- 778 events>,
 <music21.midi.MidiTrack 8 -- 1558 events>,
 <music21.midi.MidiTrack 9 -- 1338 events>,
 <music21.midi.MidiTrack 10 -- 11792 events>,
 <music21.midi.MidiTrack 11 -- 8496 events>,
 <music21.midi.MidiTrack 12 -- 1672 events>,
 <music21.midi.MidiTrack 13 -- 6474 events>,
 <music21.midi.MidiTrack 14 -- 394 events>]

In [8]:
instruments = [part.partAbbreviation for part in m21.instrument.partitionByInstrument(orig_song).parts]

In [9]:
instruments

['Str', 'Hp', 'Hn', None, 'Timp', 'Ch']

In [10]:
instrument_association = {
    'Str': m21.instrument.StringInstrument(),
    'Hp': m21.instrument.Harp(),
    'Hn': m21.instrument.Horn(),
    'Timp': m21.instrument.Timpani(),
    'Ch': m21.instrument.Choir(),
    'Fl': m21.instrument.Flute(),
    'Ob': m21.instrument.Oboe(),
    'Cl': m21.instrument.Clarinet(),
    'Bn': m21.instrument.Bassoon(),
    'Sax': m21.instrument.Saxophone(),
    'Tp': m21.instrument.Trumpet(),
    'Tbn': m21.instrument.Trombone(),
    'Tba': m21.instrument.Tuba(),
    'Perc': m21.instrument.Percussion(),
    'Vln': m21.instrument.Violin(),
    'Vla': m21.instrument.Viola(),
    'Pno': m21.instrument.Piano(),
    # None is for the case where the instrument is not specified
    # and will use any kind of percussion instrument
    None: m21.instrument.Percussion()
}

In [11]:
measures = {}
for index, inst in enumerate(m21.instrument.partitionByInstrument(orig_song).parts):
    inst.makeMeasures(inPlace=True)
    inst.makeTies(inPlace=True)
    inst.stripTies(inPlace=True, matchByPitch=True)
    if (inst.partAbbreviation == 'Hn'):
        inst.show('text')
    for measure in inst.getElementsByClass(m21.stream.Measure):
        
        if measure.offset not in measures:
            measures[measure.offset] = []
        while len(measures[measure.offset]) <= index:
            measures[measure.offset].append(None)
        measures[measure.offset][index] = Measure(measure, inst.partAbbreviation)

{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.instrument.Horn 'Staff-10: Horn'>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.tempo.MetronomeMark Quarter=192.0>
    {0.0} <music21.tempo.MetronomeMark Quarter=192.0>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Rest half>
    {0.0} <music21.note.Rest half>
    {2.0} <music21.chord.Chord E4 F#4 C#5>
    {2.0} <music21.chord.Chord E3 F#3 A3 C#4>
{4.0} <music21.stream.Measure 2 offset=4.0>
    {0.0} <music21.chord.Chord G#4 A4 E5>
    {0.0} <music21.chord.Chord G#3 A3 C#4 E4>
    {2.0} <music21.chord.Chord F#4 G#4 E-5>
    {2.0} <music21.chord.Chord F#3 G#3 B3 E-4>
{8.0} <music21.stream.Measure 3 offset=8.0>
    {0.0} <music21.chord.Chord G#4 B4 F#5>
    {0.0} <music21.chord.Chord G#3 A3 C#4 E4>
{12.0} <music21.stream.Measure 4 offset=12.0>

{16.0} <music21.stream.Measure 5 offset=16.0>
    {2.0} <music21.chord.Chord G#4 B4 F#5>
    {2.0} <music21.chord.Cho

In [12]:
def add_measure(song_, offset):
    # get the number of measures in the song
    measure_num = len(song_.parts[0].getElementsByClass(m21.stream.Measure))
    measures_in_order = measures[offset]
    for index, measure in enumerate(measures_in_order):
        if measure is None:
            continue
        if index == 0 and measure_num == 0:
            parts[index].insert(0, instrument_association[instrument])
        newMeasure = copy.deepcopy(measure.measure)
        parts[index].insert(measure_num * 4, newMeasure)
    return song_

song = m21.stream.Score()
parts = [m21.stream.Part() for _ in range(len(instrument_association))]
for index, instrument in enumerate(instruments):
    song.insert(0, parts[index])

add_measure(song, 0)
song = m21.midi.translate.prepareStreamForMidi(song)
tup = m21.midi.translate.channelInstrumentData(song)
# get all midi events from the song
song.makeMeasures(inPlace=True)
midi_tracks = m21.midi.translate.streamHierarchyToMidiTracks(song)
midi_events = [midi_track.events for midi_track in midi_tracks]
tup[0]


{48: 1, 46: 6, 60: 8, 47: 11, 52: 13, None: 2}

In [13]:
int.from_bytes(midi_events[0][1].data)

312500

In [14]:
inv_map = {v: k for k, v in tup[0].items()}

In [15]:
measures_list_ = list(measures.values())

In [16]:
idx = 0
for index, measure in enumerate(measures_list_):
    idx = index
    if len(measure) != len(instruments):
        break

idx

352

In [17]:
measures_list = measures_list_[:idx]

In [18]:
# generate a similarity matrix for the measures
similarity_matrix = np.zeros((len(measures_list), len(measures_list)))
for i in range(len(measures_list)):
    print(f'Calculating similarity for measure {i}\n')
    for j in range(len(measures_list)):
        if i == j:
            similarity_matrix[i][j] = 1
            continue
        if i > j:
            similarity_matrix[i][j] = similarity_matrix[j][i]
            continue
        # calculate the similarity between two measures
        # by calculating the levenstein distance between
        # the two measures in every instrument 
        # and then averaging the distance
        # to do so, we need to convert the measures to strings
        # and then calculate the levenstein distance
        # between the two strings
        measure1 = measures_list[i]
        measure2 = measures_list[j]
        if len(measure1) < len(measure2):
            measure1, measure2 = measure2, measure1
        total_distance = 1
        for index, measure in enumerate(measure1):
            orig_measure = measure
            measure_to_compare = measure2[index] if index < len(measure2) else None
            old_stdout = sys.stdout  

            result = StringIO()
            sys.stdout = result
            orig_measure.measure.show('text') if orig_measure is not None else None
            orig_measure_string = result.getvalue()
            
            result = StringIO()
            sys.stdout = result
            measure_to_compare.measure.show('text') if measure_to_compare is not None else None
            measure_to_compare_string = result.getvalue()

            sys.stdout = old_stdout
            
            if orig_measure is None or measure_to_compare is None:
                continue
            if orig_measure is None:
                total_distance += len()
                continue
            if measure_to_compare is None:
                total_distance += len()
                continue
            
            total_distance += lev.distance(orig_measure_string, measure_to_compare_string)
        similarity_matrix[i][j] = total_distance

# the similarity matrix now has the distance between every measure
# transform the distance in each cell to a similarity
# by subtracting the distance from the maximum distance
# and then dividing by the maximum distance
similarity_matrix = 1 / similarity_matrix
similarity_matrix
        

Calculating similarity for measure 0

Calculating similarity for measure 1

Calculating similarity for measure 2

Calculating similarity for measure 3

Calculating similarity for measure 4

Calculating similarity for measure 5

Calculating similarity for measure 6

Calculating similarity for measure 7

Calculating similarity for measure 8

Calculating similarity for measure 9

Calculating similarity for measure 10

Calculating similarity for measure 11

Calculating similarity for measure 12

Calculating similarity for measure 13

Calculating similarity for measure 14

Calculating similarity for measure 15

Calculating similarity for measure 16

Calculating similarity for measure 17

Calculating similarity for measure 18

Calculating similarity for measure 19

Calculating similarity for measure 20

Calculating similarity for measure 21

Calculating similarity for measure 22

Calculating similarity for measure 23

Calculating similarity for measure 24

Calculating similarity for measure 

array([[1.00000000e+00, 3.77073906e-04, 5.54016620e-04, ...,
        4.43655723e-04, 4.84027106e-04, 5.15198351e-04],
       [3.77073906e-04, 1.00000000e+00, 4.33275563e-04, ...,
        3.98565165e-04, 4.13393964e-04, 4.26985482e-04],
       [5.54016620e-04, 4.33275563e-04, 1.00000000e+00, ...,
        7.18390805e-04, 7.91765637e-04, 8.48176421e-04],
       ...,
       [4.43655723e-04, 3.98565165e-04, 7.18390805e-04, ...,
        1.00000000e+00, 1.36612022e-03, 1.25470514e-03],
       [4.84027106e-04, 4.13393964e-04, 7.91765637e-04, ...,
        1.36612022e-03, 1.00000000e+00, 1.56494523e-03],
       [5.15198351e-04, 4.26985482e-04, 8.48176421e-04, ...,
        1.25470514e-03, 1.56494523e-03, 1.00000000e+00]])

In [19]:
# get the most similar measure to the first measure
# by getting the index of the maximum value in the first row
# and then getting the measure at that index
most_similar_measure_index = np.argmax(similarity_matrix[0])
most_similar_measure_index

0

In [20]:
# convert the similarity matrix to a transition matrix
# where each cell in row i is the probability to transition from measure i to measure j
# and the probability to transition to the same measure is 0
# the most likely transition should be to the next measure
# so the probability to transition to the next measure should be the highest
# and the rest of the probabilities should be distributed according to the similarity to the next measure
# so the probability to transition from measure i to measure j should be the similarity between measure i + 1 and measure j

# the transition matrix should be a square matrix with the same size as the similarity matrix
transition_matrix = np.zeros((len(similarity_matrix), len(similarity_matrix)))
for i in range(len(similarity_matrix)):
    for j in range(len(similarity_matrix)):
        if i == j:
            continue
        transition_matrix[i][j] = similarity_matrix[j][i + 1]  if i + 1 < len(similarity_matrix) else similarity_matrix[j][0]
    transition_matrix[i] = transition_matrix[i] / np.sum(transition_matrix[i])



In [21]:
transition_matrix

array([[0.00000000e+00, 8.62794834e-01, 3.73827918e-04, ...,
        3.43879966e-04, 3.56674177e-04, 3.68400869e-04],
       [4.20959444e-04, 0.00000000e+00, 7.59831796e-01, ...,
        5.45856175e-04, 6.01608706e-04, 6.44471413e-04],
       [3.77201194e-04, 3.08014105e-04, 0.00000000e+00, ...,
        4.47069908e-04, 4.94486413e-04, 5.32739798e-04],
       ...,
       [3.55035853e-04, 3.03226156e-04, 5.80763319e-04, ...,
        0.00000000e+00, 7.33504072e-01, 1.14789370e-03],
       [3.70908464e-04, 3.07401080e-04, 6.10630474e-04, ...,
        9.03304051e-04, 0.00000000e+00, 7.19933328e-01],
       [8.63841971e-01, 3.25732267e-04, 4.78582809e-04, ...,
        3.83248434e-04, 4.18122929e-04, 0.00000000e+00]])

In [22]:
# save the transition matrix to a file
np.savetxt('transition_matrix.csv', transition_matrix, delimiter=',')

In [23]:
# load the transition matrix from a file
transition_matrix = np.loadtxt('transition_matrix.csv', delimiter=',')

In [24]:
def new_add_measure(song_, previous_index):
    # get the number of measures in the song
    measure_num = len(song_.parts[0].getElementsByClass(m21.stream.Measure))
    # use the transition matrix to get the next measure
    next_measure_index = np.random.choice(len(transition_matrix[previous_index]), p=transition_matrix[previous_index])
    measures_in_order = measures_list[next_measure_index]
    for index, measure in enumerate(measures_in_order):
        if measure is None:
            continue
        if index == 0 and measure_num == 0:
            parts[index].insert(0, instrument_association[instrument])
        newMeasure = copy.deepcopy(measure.measure)
        parts[index].insert(measure_num * 4, newMeasure)
    return song_, next_measure_index

In [25]:
song = m21.stream.Score()
parts = [m21.stream.Part() for _ in range(len(instrument_association))]
for index, instrument in enumerate(instruments):
    song.insert(0, parts[index])

current_new_measure = 0
for i in range(len(measures_list)):
    song, current_new_measure = new_add_measure(song, current_new_measure)
    print(f'Added measure {current_new_measure}')
song.write('midi', fp='test.mid')

Added measure 1
Added measure 195
Added measure 196
Added measure 197
Added measure 34
Added measure 35
Added measure 36
Added measure 37
Added measure 38
Added measure 39
Added measure 40
Added measure 41
Added measure 325
Added measure 326
Added measure 327
Added measure 328
Added measure 329
Added measure 330
Added measure 331
Added measure 16
Added measure 17
Added measure 18
Added measure 19
Added measure 109
Added measure 232
Added measure 188
Added measure 189
Added measure 190
Added measure 191
Added measure 192
Added measure 193
Added measure 194
Added measure 195
Added measure 196
Added measure 321
Added measure 322
Added measure 323
Added measure 324
Added measure 325
Added measure 326
Added measure 327
Added measure 328
Added measure 329
Added measure 330
Added measure 250
Added measure 251
Added measure 252
Added measure 253
Added measure 254
Added measure 141
Added measure 142
Added measure 143
Added measure 144
Added measure 145
Added measure 226
Added measure 227
Added 

'test.mid'

In [26]:
# write measures_list to a file with pickle
with open('measures_list.pkl', 'wb') as f:
    pickle.dump(measures_list, f)

In [27]:
# read measures_list from a file with pickle
with open('measures_list.pkl', 'rb') as f:
    measures_list = pickle.load(f)

In [28]:
len(measures_list)

352